<a href="https://colab.research.google.com/github/tlsgptj/CJdaehan/blob/main2/CJ_%EB%8C%80%ED%95%9C%ED%86%B5%EC%9A%B4_%EB%AC%BC%EB%A5%98_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install tensorflow scikit-learn

In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 이미지 데이터 경로와 라벨링 정보 파일 경로를 설정합니다.
data_folder_path = "/content/drive/MyDrive/sangpum_image"  # 이미지 파일과 라벨링 정보가 같은 폴더에 있는 디렉토리

# 라벨을 숫자로 인코딩하기 위한 LabelEncoder 객체를 생성합니다.
label_encoder = LabelEncoder()

# 이미지와 라벨을 저장할 리스트를 초기화합니다.
images = []
labels = []

# 이미지 파일과 라벨링 정보 파일을 하나씩 처리합니다.
for file_name in os.listdir(data_folder_path):
    if file_name.endswith(".jpg"):
        image_path = os.path.join(data_folder_path, file_name)

        # 이미지를 로드하여 넘파이 배열로 변환합니다.
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
        image = tf.keras.preprocessing.image.img_to_array(image)

        # 이미지 파일명에서 라벨링 정보 파일의 경로를 생성합니다.
        xml_file_name = file_name.replace(".jpg", ".xml")
        xml_file_path = os.path.join(data_folder_path, xml_file_name)

        # .xml 파일을 파싱하여 객체 정보를 추출합니다.
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        for obj in root.findall("object"):
            label = obj.find("name").text

            # 이미지와 라벨을 리스트에 추가합니다.
            images.append(image)
            labels.append(label)

# 라벨을 숫자로 인코딩합니다.
labels = label_encoder.fit_transform(labels)

# 이미지 데이터를 넘파이 배열로 변환합니다.
images = np.array(images)
labels = np.array(labels)

# 데이터를 학습용과 테스트용으로 나눕니다.
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# CNN 모델 정의
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 학습된 모델을 저장합니다.
model.save("object_detection_model.h5")

# 학습된 모델을 로드합니다.
loaded_model = tf.keras.models.load_model("object_detection_model.h5")

# 이미지를 전처리하고 모델을 통해 예측합니다.
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

# 테스트 이미지를 지정합니다.
test_image_path = "/content/drive/MyDrive/sangpum_image/img_11404.jpg"

# 테스트 이미지를 전처리하고 모델을 통해 예측합니다.
preprocessed_image = preprocess_image(test_image_path)
predictions = loaded_model.predict(preprocessed_image)
predicted_label = label_encoder.classes_[np.argmax(predictions)]
print("Predicted Label:", predicted_label)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


35/35 [==============================] - 147s 4s/step - loss: 42.3072 - accuracy: 0.0155 - val_loss: 4.5912 - val_accuracy: 0.0327
Epoch 2/10
35/35 [==============================] - 143s 4s/step - loss: 4.5875 - accuracy: 0.0347 - val_loss: 4.5868 - val_accuracy: 0.0327
Epoch 3/10
35/35 [==============================] - 141s 4s/step - loss: 4.5816 - accuracy: 0.0347 - val_loss: 4.5826 - val_accuracy: 0.0327
Epoch 4/10
35/35 [==============================] - 140s 4s/step - loss: 4.5759 - accuracy: 0.0347 - val_loss: 4.5789 - val_accuracy: 0.0327
Epoch 5/10
35/35 [==============================] - 153s 4s/step - loss: 4.5705 - accuracy: 0.0347 - val_loss: 4.5749 - val_accuracy: 0.0327
Epoch 6/10
35/35 [==============================] - 143s 4s/step - loss: 4.5652 - accuracy: 0.0347 - val_loss: 4.5716 - val_accuracy: 0.0327
Epoch 7/10
35/35 [==============================] - 143s 4s/step - loss: 4.5600 - accuracy: 0.0347 - val_loss: 4.5679 - val_accuracy: 0.0327
Epoch 8/10
35/35 [=====

In [ ]:
!pip install tensorflow
!pip install tensorflow-object-detection-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.7 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844490 sha256=c51b969ea54a8cfa2b5f5bed7aaec267b719d0e1cab2f2ace3bdc2a831272728
  Stored in directory: /root/.cache/pip/wheels/8a/55/68/c084bc2cd93c41fd8f7e2ef9e6bbcb2c35a3e4b49e42044d02
Successfully built tensorflow-object-detection-api


In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import matplotlib.pyplot as plt

# 구글 드라이브와 연동합니다.
from google.colab import drive
drive.mount('/content/drive')

# 이미지 데이터 경로와 라벨링 정보 파일 경로를 설정합니다.
data_folder_path = "/content/drive/MyDrive/sangpum_image"  # 이미지 파일과 라벨링 정보가 같은 폴더에 있는 디렉토리
label_map_path = "/content/label_map.pbtxt"

# 라벨을 숫자로 인코딩하기 위한 LabelEncoder 객체를 생성합니다.
label_encoder = LabelEncoder()

# 이미지와 라벨을 저장할 리스트를 초기화합니다.
images = []
labels = []

# 이미지 파일과 라벨링 정보 파일을 하나씩 처리합니다.
for file_name in os.listdir(data_folder_path):
    if file_name.endswith(".jpg"):
        image_path = os.path.join(data_folder_path, file_name)

        # 이미지를 로드하여 넘파이 배열로 변환합니다.
        image_np = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
        image_np = tf.keras.preprocessing.image.img_to_array(image_np)

        # 파일명에서 라벨링 정보 파일(.xml)의 경로를 생성합니다.
        xml_file_name = file_name.replace(".jpg", ".xml")
        xml_file_path = os.path.join(data_folder_path, xml_file_name)

        # .xml 파일을 파싱하여 객체 정보를 추출합니다.
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        for obj in root.findall("object"):
            label = obj.find("name").text

            # 이미지와 라벨을 리스트에 추가합니다.
            images.append(image_np)
            labels.append(label)

# 라벨을 숫자로 인코딩합니다.
labels = label_encoder.fit_transform(labels)

# 이미지 데이터를 넘파이 배열로 변환합니다.
images = np.array(images)
labels = np.array(labels)

# 데이터를 학습용과 테스트용으로 나눕니다.
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# CNN 모델 정의
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_test, y_test))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


18/18 [==============================] - 6s 147ms/step - loss: 78.7681 - accuracy: 0.0073 - val_loss: 4.6586 - val_accuracy: 0.0145
Epoch 2/30
18/18 [==============================] - 2s 130ms/step - loss: 4.2294 - accuracy: 0.0894 - val_loss: 3.9345 - val_accuracy: 0.1673
Epoch 3/30
18/18 [==============================] - 2s 109ms/step - loss: 2.6102 - accuracy: 0.4443 - val_loss: 2.7620 - val_accuracy: 0.5273
Epoch 4/30
18/18 [==============================] - 2s 118ms/step - loss: 1.2358 - accuracy: 0.7564 - val_loss: 2.3356 - val_accuracy: 0.6182
Epoch 5/30
18/18 [==============================] - 2s 97ms/step - loss: 0.6988 - accuracy: 0.8723 - val_loss: 2.2433 - val_accuracy: 0.6691
Epoch 6/30
18/18 [==============================] - 2s 99ms/step - loss: 0.5587 - accuracy: 0.8878 - val_loss: 1.9317 - val_accuracy: 0.7127
Epoch 7/30
18/18 [==============================] - 2s 98ms/step - loss: 0.4491 - accuracy: 0.9051 - val_loss: 1.8219 - val_accuracy: 0.7127
Epoch 8/30
18/18 [=

In [ ]:
model.save("Future_AI_6924.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

# 저장된 모델을 로드합니다.
loaded_model = tf.keras.models.load_model("Future_AI_6924.h5")

# 테스트 데이터셋에 대한 예측을 수행합니다.
y_pred_probabilities = loaded_model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probabilities, axis=1)

# 예측 결과를 정확도로 평가합니다.
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")

9/9 [==============================] - 0s 23ms/step
Test Accuracy: 0.7164


In [ ]:
label_map = {
    '불닭볶음면 할라피뇨치즈' : 1,
    '불닭볶음면 화끈한매운맛' : 2,
    '게토레이 레몬향' : 3,
    '코카콜라' : 4,
    '스낵면' : 5,
    '몽실몽실 복숭아 아이스티' : 6,
    '웰치스 그레이프맛' : 7,
    '내커피색깔은블랙' : 8,
    '밀키스 캔' : 9,
    '생생우동면' : 10,
    '까스활명수' : 11,
    '신라면 컵 15개입' : 12,
    '깨끗한나라 휴지 2개' : 13,
    '기네스 extra stout dark lively' : 15,
    '필스너우르켈캔' : 16,
    '짜파게티 큰사발면' : 17,
    '까르보 불닭볶음면' : 18,
    '기린 이치방' : 19,
    '탐스제로레몬' : 20,
    '상큼달콤 요걸리' : 21,
    '후루룩 쌀국수 미역국' : 23,
    '청하_연하늘색' : 24,
    '청하_진파랑색' : 25,
    '볶음 간짬뽕 큰컵' : 26,
    '속청 쿨' : 29,
    '웅진 초록매실' : 30,
    '스위트 아메리카노' : 31,
    '말표 맥주 캔' : 32 ,
    '프레첼 갈릭버터맛' : 34,
    '솔의눈 pet' : 35,
    '알로에베라킹 프리미엄 제로' : 37,
    '콘트라베이스 black&shot' : 38,
    '짜파구리 매운맛' : 40,
    '팔도비빔면 컵' : 41,
    '치킨커리 우동' : 42,
    '메밀 비빔막국수' : 43,
    '게토레이블루' : 44,
    '포도 봉봉' : 45,
    '칸타타 아이스 스위트 아메리카노' : 46,
    '코카콜라제로' : 47,
    '크리넥스 안심 키친타월' : 48,
    '팔도 김치 도시락' : 49,
    '농심 신볶게티' : 50,
    '컨디션 lady' : 51,
    '하이트 제로 논알콜 캔' : 52,
    '미닛메이드 오렌지' : 53,
    '닥터유 단백질 초코' : 54,
    '종근당 산에는 삼' : 55,
    '박카스f' : 56,
    '칸타타 카라멜마키아토' : 57,
    '비상대책 숙취해소' : 58,
    '코카콜라 zero sugar 라임맛' : 59,
    '맥콜 제로' : 60,
    '코카콜라 오리지널 pet' : 61,
    '비락 식혜 pet' : 62,
    '콘소메맛 팝콘' : 63,
    '초코칩쿠키' : 64,
    '팔도 왕뚜껑' : 65,
    '칸타타 아이스 블랙커피' : 67,
    '양 맛있는 두유 gt' : 68,
    '롯데 칠성 사이다 캔' : 69,
    '롯데 수박바 에이드' : 70,
    '농심 직화쌀짬뽕 컵' : 72,
    '진로 토닉 제로' : 73,
    '롤티슈 득템' : 74,
    '농심 튀김우동 컵' : 75,
    '농심 오징어짬뽕 큰사발면' : 76,
    '펩시 콜라 캔' : 77,
    '박카스 디카페' : 79,
    '크루저 블루베리 병' : 80,
    '오뚜기 콕콕콕 스파게티 용기면' : 81,
    '곰표 오리지널 나쵸' : 82,
    '초코에몽 드링크 ice' : 83,
    '마스터 바닐라블랙 pet' : 84,
    '월매 쌀막걸리' : 85,
    '포카리스웨트 pet' : 86,
    '칸타타 스위트 아메리카노 캔' : 87,
    '농심 오징어짬뽕 컵' : 88,
    '파울라너 바이스비어 캔' : 89,
    '오뚜기 컵누들 매콤한 맛' : 90,
    '오뚜기 육개장 big' : 91,
    '하이트 extra cold 캔' : 92,
    '뽀로로 딸기맛 pet' : 93,
    '농심 육개장 큰사발면' : 95,
    '프레첼 체다치즈맛' : 96,
    '서든어택 펑 캔' : 97,
    '나무야나무야 클린3겹 24롤' : 98,
    '웅진 행복가득세트' : 99,
    '참이슬 fresh' : 100,
    '하늘보리 pet' : 101,
    '맥심 에스프레소 top 마스터 라떼 캔' : 102,
    'tealog 납작 복숭아 아이스티 제로' : 103,
    '농심 튀김우동 큰사발면'	: 105,
    '농심 사리곰탕 큰사발면'	: 107,
    '팔도 짬뽕 왕뚜껑'	: 108,
    '오뚜기 진라면 매운맛'	: 109,
    '오뚜기 콕콕콕 마요짜장볶이'	: 110,
    '얼큰 유부우동 한그릇'	: 111,
    '오뚜기 열 참깨라면'	: 113,
    '오뚜기 콕콕콕 치즈볶이'	: 114
}

with open('label_map.pbtxt', 'w') as f:
    for class_name, class_id in label_map.items():
        f.write("item {\n")
        f.write(f"  id: {class_id}\n")
        f.write(f"  name: '{class_name}'\n")
        f.write("}\n")


In [ ]:
import os
import numpy as np
import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import matplotlib.pyplot as plt

# 구글 드라이브와 연동합니다.
from google.colab import drive
drive.mount('/content/drive')

# 이미지 데이터 경로와 라벨링 정보 파일 경로를 설정합니다.
data_folder_path = "/content/drive/MyDrive/sangpum_image"  # 이미지 파일과 라벨링 정보가 같은 폴더에 있는 디렉토리
label_map_path = "/content/label_map.pbtxt"

# 라벨 맵 파일을 읽어옵니다.
label_map = label_map_util.load_labelmap(tf.io.gfile.GFile(label_map_path))
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=100, use_display_name=True)
category_index = label_map_util.create_category_index(categories)



custom_model_path = "/content/Future_AI_6924.h5"
custom_model = tf.keras.models.load_model(custom_model_path)

# 모델 추론을 위한 함수 정의
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # 이미지를 넘파이 배열로 변환합니다.
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    # 모델 추론을 수행합니다.
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # 추론 결과를 딕셔너리 형태로 반환합니다.
    return output_dict

# 이미지와 라벨을 저장할 리스트를 초기화합니다.
images = []
labels = []

# 이미지 파일과 라벨링 정보 파일을 하나씩 처리합니다.
for file_name in os.listdir(data_folder_path):
    if file_name.endswith(".jpg"):
        image_path = os.path.join(data_folder_path, file_name)

        # 이미지를 로드하여 넘파이 배열로 변환합니다.
        image_np = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
        image_np = tf.keras.preprocessing.image.img_to_array(image_np)

        # 모델 추론을 수행합니다.
        output_dict = run_inference_for_single_image(model, image_np)

        # 추론 결과를 이용하여 객체 검출 결과를 시각화합니다.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.array(output_dict['detection_boxes'][0]),
            np.array(output_dict['detection_classes'][0]).astype(np.int32),
            np.array(output_dict['detection_scores'][0]),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)

        # 이미지와 라벨을 리스트에 추가합니다.
        images.append(image_np)
        labels.append(file_name)

# 시각화된 이미지를 출력합니다.
for i, image_np in enumerate(images):
    plt.figure(figsize=(10, 10))
    plt.imshow(image_np)
    plt.axis('off')
    plt.title(f'Image: {labels[i]}')
    plt.show()


ModuleNotFoundError: ignored

In [ ]:
# 학습된 모델을 로드합니다.
loaded_model = tf.keras.models.load_model("object_detection_model.h5")

# 이미지를 전처리하고 모델을 통해 예측합니다.
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

# 테스트 이미지를 지정합니다.
test_image_path = "/content/drive/MyDrive/sangpum_image/img_11404.jpg"

# 테스트 이미지를 전처리하고 모델을 통해 예측합니다.
preprocessed_image = preprocess_image(test_image_path)
detections = loaded_model.predict(preprocessed_image)

# 결과를 시각화하여 객체와 라벨을 확인합니다.
vis_util.visualize_boxes_and_labels_on_image_array(
    preprocessed_image[0],
    detections['detection_boxes'][0],
    detections['detection_classes'][0].astype(np.int32),
    detections['detection_scores'][0],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.2,
    line_thickness=5
)

# 시각화된 이미지를 출력합니다.
plt.imshow(preprocessed_image[0])
plt.show()

1/1 [==============================] - 0s 124ms/step


IndexError: ignored